# __Ensemble Algorithms__

## __Bootstrap Aggregation__

Decision trees are a simple and powerful predictive modeling technique, but they suﬀer from
**high-variance. This means that trees can get very diﬀerent results given diﬀerent training data**.
A technique to make decision trees more robust and to achieve better performance is called
bootstrap aggregation or bagging for short.

A bootstrap is a sample of a dataset with replacement. This means that a new dataset is created
from a random sample of an existing dataset where a given row may be selected and added
more than once to the sample. It is a useful approach to use when estimating values such as
the mean for a broader dataset, when you only have a limited dataset available. By creating
samples of your dataset and estimating the mean from those samples, you can take the average
of those estimates and get a better idea of the true mean of the underlying problem.
This same approach can be used with machine learning algorithms that have a high variance.

**A separate model is trained on each bootstrap sample of data and the average output of those models used to make predictions.
This technique is called bootstrap aggregation or bagging** for short. Variance means that an
algorithm’s performance is sensitive to the training data, with __high variance suggesting that the
more the training data is changed, the more the performance of the algorithm will vary__.

The performance of high variance machine learning algorithms like unpruned decision trees
can be improved by training many trees and taking the average of their predictions. Results are
often better than a single decision tree. **Another benefit of bagging in addition to improved
performance is that the bagged decision trees cannot overfit the problem. Trees can continue to
be added until a maximum in performance is achieved.**

### Bootstrap Resample

We can create a
new sample of a dataset by randomly selecting rows from the dataset and adding them to a new
list. We can repeat this for a fixed number of rows or until the size of the new dataset matches
a ratio of the size of the original dataset.
We can allow sampling with replacement by not removing the row that was selected so that
it is available for future selections. 

In [59]:
# Example of subsampling a dataste
from random import seed
from random import randrange
from math import sqrt
from math import exp
from csv import reader

In [13]:
# Create a random subsample from the dataset with replacement
def subsample(dataset, ratio=1.0):
    sample = list()
    n_sample = round(len(dataset) * ratio)
    while len(sample) < n_sample:
        index = randrange(len(dataset))
        sample.append(dataset[index])
    return sample

In [29]:
# Calculate the mean of a list of numbers EXAMPLE
def mean(numbers):
    return sum(numbers) / float(len(numbers))
    
# Test subsampling a dataset
seed(1)

# True mean
dataset = [[randrange(10)] for i in range(20)]
print('True Mean: %.3f' % mean([row[0] for row in dataset]))

# Estimated means
ratio = 0.10
for size in [1, 10, 100]:
    sample_means = list()
    for i in range(size):
        sample = subsample(dataset, ratio)
        sample_mean = mean([row[0] for row in sample])
        sample_means.append(sample_mean)
    print('Samples=%d, Estimated Mean: %.3f' % (size, mean(sample_means)))

True Mean: 4.500
Samples=1, Estimated Mean: 4.000
Samples=10, Estimated Mean: 4.700
Samples=100, Estimated Mean: 4.570


### Sonar Case Study

In [31]:
# Bagging Algorithm on the Sonar dataset
from random import seed
from random import randrange
from csv import reader

# Load a CSV file
def load_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset

# Convert string column to float
def str_column_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column].strip())
        
# Convert string column to integer
def str_column_to_int(dataset, column):
    class_values = [row[column] for row in dataset]
    unique = set(class_values)
    lookup = dict()
    for i, value in enumerate(unique):
        lookup[value] = i
    for row in dataset:
        row[column] = lookup[row[column]]
    return lookup
    
# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for _ in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split
    
# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
    return scores
    
# Split a dataset based on an attribute and an attribute value
def test_split(index, value, dataset):
    left, right = list(), list()
    for row in dataset:
        if row[index] < value:
            left.append(row)
        else:
            right.append(row)
    return left, right
    
# Calculate the Gini index for a split dataset
def gini_index(groups, classes):
    # count all samples at split point
    n_instances = float(sum([len(group) for group in groups]))
    # sum weighted Gini index for each group
    gini = 0.0
    for group in groups:
        size = float(len(group))
        # avoid divide by zero
        if size == 0:
            continue
        score = 0.0
        # score the group based on the score for each class
        for class_val in classes:
            p = [row[-1] for row in group].count(class_val) / size
            score += p * p
        # weight the group score by its relative size
        gini += (1.0 - score) * (size / n_instances)
    return gini

# Select the best split point for a dataset
def get_split(dataset):
    class_values = list(set(row[-1] for row in dataset))
    b_index, b_value, b_score, b_groups = 999, 999, 999, None
    for index in range(len(dataset[0])-1):
        for row in dataset:
        # for i in range(len(dataset)):
            # row = dataset[randrange(len(dataset))]
            groups = test_split(index, row[index], dataset)
            gini = gini_index(groups, class_values)
            if gini < b_score:
                b_index, b_value, b_score, b_groups = index, row[index], gini, groups
    return {'index':b_index, 'value':b_value, 'groups':b_groups}

# Create a terminal node value
def to_terminal(group):
    outcomes = [row[-1] for row in group]
    return max(set(outcomes), key=outcomes.count)
    
# Create child splits for a node or make terminal
def split(node, max_depth, min_size, depth):
    left, right = node['groups']
    del(node['groups'])
    
    # check for a no split
    if not left or not right:
        node['left'] = node['right'] = to_terminal(left + right)
        return  

    # check for max depth
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    
    # process left child
    if len(left) <= min_size or len(set([row[-1] for row in left]))==1:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split(left)       
        split(node['left'], max_depth, min_size, depth+1)
        
    # process right child
    if len(right) <= min_size or len(set([row[-1] for row in right]))==1:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right)
        split(node['right'], max_depth, min_size, depth+1)

# Build a decision tree
def build_tree(train, max_depth, min_size):
    root = get_split(train)
    split(root, max_depth, min_size, 1)
    return root

# Make a prediction with a decision tree
def predict(node, row):
    if row[node['index']] < node['value']:
        if isinstance(node['left'], dict):
            return predict(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row)
        else:
            return node['right']

# Make a prediction with a list of bagged trees
def bagging_predict(trees, row):
    predictions = [predict(tree, row) for tree in trees]
    return max(set(predictions), key=predictions.count)

# Bootstrap Aggregation Algorithm
def bagging(train, test, max_depth, min_size, sample_size, n_trees):
    trees = list()
    for _ in range(n_trees):
        sample = subsample(train, sample_size)
        tree = build_tree(sample, max_depth, min_size)
        trees.append(tree)
    predictions = [bagging_predict(trees, row) for row in test]
    return(predictions)
   

In [34]:
# Test bagging on the sonar dataset
seed(1)
# load and prepare data
filepath = '/Users/maheshwars/Desktop/venv/data/sonar'
filename = filepath +'/sonar_data.csv'
dataset = load_csv(filename)
print('Loaded data file {0} with {1} rows and {2} columns'.format(filename, len(dataset),
                                                                  len(dataset[0])))

# convert string attributes to integers
for i in range(len(dataset[0])-1):
    str_column_to_float(dataset, i)
    
# convert class column to integers
str_column_to_int(dataset, len(dataset[0])-1)

# evaluate algorithm
n_folds = 5
max_depth = 6
min_size = 2
sample_size = 0.50

for n_trees in [1, 5, 10, 50,100]:
    scores = evaluate_algorithm(dataset, bagging, n_folds, max_depth, min_size, sample_size, n_trees)
    print('Trees: %d' % n_trees)
    print('Scores: %s' % scores)
    print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

Loaded data file /Users/maheshwars/Desktop/venv/data/sonar/sonar_data.csv with 208 rows and 61 columns
Trees: 1
Scores: [87.8048780487805, 65.85365853658537, 65.85365853658537, 65.85365853658537, 73.17073170731707]
Mean Accuracy: 71.707%
Trees: 5
Scores: [60.97560975609756, 80.48780487804879, 78.04878048780488, 82.92682926829268, 63.41463414634146]
Mean Accuracy: 73.171%
Trees: 10
Scores: [58.536585365853654, 78.04878048780488, 80.48780487804879, 87.8048780487805, 65.85365853658537]
Mean Accuracy: 74.146%
Trees: 50
Scores: [65.85365853658537, 75.60975609756098, 80.48780487804879, 73.17073170731707, 82.92682926829268]
Mean Accuracy: 75.610%
Trees: 100
Scores: [85.36585365853658, 87.8048780487805, 75.60975609756098, 73.17073170731707, 73.17073170731707]
Mean Accuracy: 79.024%


__A diﬃculty of this method is that even though deep trees are constructed, the bagged trees
that are created are very similar. In turn, the predictions made by these trees are also similar,
and the high variance we desire among the trees trained on diﬀerent samples of the training
dataset is diminished.
This is because of the greedy algorithm used in the construction of the trees selecting the
same or similar split points.__

# **Random Forest**

Decision trees involve the greedy selection of the best split point from the dataset at each step.
This algorithm makes decision trees susceptible to high variance if they are not pruned. This
high variance can be harnessed and reduced by creating multiple trees with diﬀerent samples
of the training dataset (diﬀerent views of the problem) and combining their predictions. This
approach is called bootstrap aggregation or bagging for short.

A limitation of bagging is that the same greedy algorithm is used to create each tree, meaning
that it is likely that **the same or very similar split points will be chosen in each tree** making
the diﬀerent trees very similar (trees will be correlated). This, in turn, makes their predictions
similar, mitigating the variance originally sought.

We can force the decision trees to be diﬀerent by limiting the features (columns) that the
greedy algorithm can evaluate at each split point when creating the tree. This is called the
Random Forest algorithm. Like bagging, multiple samples of the training dataset are taken and
a diﬀerent tree trained on each. The diﬀerence is that at each point a split is made in the data
and added to the tree, only a fixed subset of attributes can be considered. For classification
problems, the type of problems we will look at in this tutorial, the number of attributes to be
considered for the split is limited to the square root of the number of input features.

    num features for split = total input features         
The result of this one small change are trees that are more diﬀerent from each other
(uncorrelated) resulting in predictions that are more diverse and a combined prediction that
often has better performance than a single tree or bagging alone.

## Calculating Splits

Finding the best split point in a decision tree involves evaluating the cost of each value in
the training dataset for each input variable. For bagging and random forest, this procedure
is executed upon a sample of the training dataset, made with replacement. Sampling with
replacement means that the same row may be chosen and added to the sample more than once.
We can update this procedure for Random Forest. Instead of enumerating all values for
input attributes in search of the split with the lowest cost, we can create a sample of the input
attributes to consider. This sample of input attributes can be chosen randomly and without replacement, meaning that each input attribute needs to only be considered once when looking
for the split point with the lowest cost.

In [43]:
# Select the best split point for a dataset
def get_split(dataset, n_features):
    class_values = list(set(row[-1] for row in dataset))
    b_index, b_value, b_score, b_groups = 999, 999, 999, None
    features = list()
    while len(features) < n_features:
        index = randrange(len(dataset[0])-1)
        if index not in features:
            features.append(index)
    for index in features:
        for row in dataset:
            groups = test_split(index, row[index], dataset)
            gini = gini_index(groups, class_values)
            if gini < b_score:
                b_index, b_value, b_score, b_groups = index, row[index], gini, groups
    return {'index':b_index, 'value':b_value, 'groups':b_groups}

# Create child splits for a node or make terminal
def split(node, max_depth, min_size, n_features, depth):
    left, right = node['groups']
    del(node['groups'])
    
    # check for a no split
    if not left or not right:
        node['left'] = node['right'] = to_terminal(left + right)
        return
    # check for max depth
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    
    # process left child
    if len(left) <= min_size:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split(left, n_features)
        split(node['left'], max_depth, min_size, n_features, depth+1)
    
    # process right child
    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right, n_features)
        split(node['right'], max_depth, min_size, n_features, depth+1)

In [1]:
# Build a decision tree
def build_tree(train, max_depth, min_size, n_features): # n_features #new
    root = get_split(train, n_features)
    split(root, max_depth, min_size, n_features, 1)
    return root

# Random Forest Algorithm
def random_forest(train, test, max_depth, min_size, sample_size, n_trees, n_features):
    trees = list()
    for _ in range(n_trees):
        sample = subsample(train, sample_size)
        tree = build_tree(sample, max_depth, min_size, n_features)
        trees.append(tree)
    predictions = [bagging_predict(trees, row) for row in test]
    return(predictions)



In [47]:
# Test the random forest algorithm on sonar dataset
seed(2)
# load and prepare data
filepath = '/Users/maheshwars/Desktop/venv/data/sonar'
filename = filepath +'/sonar_data.csv'
dataset = load_csv(filename)
print('Loaded data file {0} with {1} rows and {2} columns'.format(filename, len(dataset),
                                                                  len(dataset[0])))

# convert string attributes to integers
for i in range(0, len(dataset[0])-1):
    str_column_to_float(dataset, i)
    
# convert class column to integers
str_column_to_int(dataset, len(dataset[0])-1)

# evaluate algorithm
n_folds = 5
max_depth = 10
min_size = 1
sample_size = 1.0
n_features = int(sqrt(len(dataset[0])-1))
for n_trees in [1, 5, 10]:
    scores = evaluate_algorithm(dataset, random_forest, n_folds, max_depth, min_size,
    sample_size, n_trees, n_features)
    print('Trees: %d' % n_trees)
    print('Scores: %s' % scores)
    print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

Loaded data file /Users/maheshwars/Desktop/venv/data/sonar/sonar_data.csv with 208 rows and 61 columns
Trees: 1
Scores: [56.09756097560976, 63.41463414634146, 60.97560975609756, 58.536585365853654, 73.17073170731707]
Mean Accuracy: 62.439%
Trees: 5
Scores: [70.73170731707317, 58.536585365853654, 85.36585365853658, 75.60975609756098, 63.41463414634146]
Mean Accuracy: 70.732%
Trees: 10
Scores: [82.92682926829268, 75.60975609756098, 97.5609756097561, 80.48780487804879, 68.29268292682927]
Mean Accuracy: 80.976%


# **Stacked Generalisation Algorithm**

Stacked Generalization is an ensemble algorithm where a new model is trained to __combine the
predictions from two or more models__ already trained or your dataset. The predictions from the
existing models or submodels **are combined using a new model**, and as such stacking is often
referred to as **blending**, as the predictions from submodels are blended together.

It is typical to use **a simple linear method to combine the predictions for submodels. Examples
include: simple averaging called voting, a weighted sum using linear regression. and logistic
regression.** Models that have their predictions combined must have skill on the problem, but
do not need to be the best possible models. This means that you do not need to tune the
__submodels__ intently, **as long as the model shows some advantage over a baseline prediction.
It is important that submodels produce diﬀerent predictions, so-called uncorrelated predicions.** Stacking works best when the predictions that are combined are all skillful, but skillful
in diﬀerent ways. This may be achieved by **using algorithms that use very diﬀerent internal
representations (trees compared to instances) and/or models trained on diﬀerent representations
or projections of the training data.**

## Submodels and Aggregator

We are going to use two models as submodels for stacking and a linear model as the aggregator
model.
This part is divided into 3 sections:
1. Submodel #1: k-Nearest Neighbors.
2. Submodel #2: Perceptron.
3. Aggregator Model: Logistic Regression.
   
Each model will be described in terms of the functions used to train the model and a function
used to make predictions.

### Submodel #1: k-Nearest Neighbors
The k-Nearest Neighbors algorithm or KNN uses the entire training dataset as the model.
Therefore training the model involves retaining the training dataset. Below is a function named
knn model() that does just this.

In [48]:
# Prepare the KNN model
def knn_model(train):
    return train

In [50]:
# Calculate the Euclidean distance between two vectors
def euclidean_distance(row1, row2):
    distance = 0.0
    for i in range(len(row1)-1):
        distance += (row1[i] - row2[i])**2
    return sqrt(distance)

# Locate neighbors for a new row
def get_neighbors(train, test_row, num_neighbors):
    distances = list()
    for train_row in train:
        dist = euclidean_distance(test_row, train_row)
        distances.append((train_row, dist))
    distances.sort(key=lambda tup: tup[1])
    neighbors = list()
    for i in range(num_neighbors):
        neighbors.append(distances[i][0])
    return neighbors

# Make a prediction with KNN
def knn_predict(model, test_row, num_neighbors=2):
    neighbors = get_neighbors(model, test_row, num_neighbors)
    output_values = [row[-1] for row in neighbors]
    prediction = max(set(output_values), key=output_values.count)
    return prediction

### Submodel #2: Perceptron

The model for the Perceptron algorithm is a set of weights learned from the training data. In
order to train the weights, many predictions need to be made on the training data in order
to calculate error values. Therefore, both model training and prediction require a function for
prediction.

In [51]:
# Make a prediction with weights
def perceptron_predict(model, row):
    activation = model[0] #bias
    for i in range(len(row)-1):
        activation += model[i + 1] * row[i]
    return 1.0 if activation >= 0.0 else 0.0

# Estimate Perceptron weights using stochastic gradient descent
def perceptron_model(train, l_rate=0.01, n_epoch=5000):
    weights = [0.0 for i in range(len(train[0]))]
    for epoch in range(n_epoch):
        for row in train:
            prediction = perceptron_predict(weights, row)
            error = row[-1] - prediction
            weights[0] = weights[0] + l_rate * error
            for i in range(len(row)-1):
                weights[i + 1] = weights[i + 1] + l_rate * error * row[i]
    return weights    

### Aggregator Model: Logistic Regression

In [52]:
# Make a prediction with coefficients
def logistic_regression_predict(model, row):
    yhat = model[0]
    for i in range(len(row)-1):
        yhat += model[i + 1] * row[i]
    return 1.0 / (1.0 + exp(-yhat))

# Estimate logistic regression coefficients using stochastic gradient descent
def logistic_regression_model(train, l_rate=0.01, n_epoch=5000):
    coef = [0.0 for i in range(len(train[0]))]
    for epoch in range(n_epoch):
        for row in train:
            yhat = logistic_regression_predict(coef, row)
            error = row[-1] - yhat
            coef[0] = coef[0] + l_rate * error * yhat * (1.0 - yhat)
        for i in range(len(row)-1):
            coef[i + 1] = coef[i + 1] + l_rate * error * yhat * (1.0 - yhat) * row[i]
    return coef

### Combining Predictions

For a machine learning algorithm, learning how to combine predictions is much the same as
learning from a training dataset. A new training dataset can be constructed from the predictions
of the submodels, as follows:

* Each row represents one row in the training dataset.
* The first column contains predictions for each row in the training dataset made by the
first submodel, such as k-Nearest Neighbors.
* The second column contains predictions for each row in the training dataset made by the
second submodel, such as the Perceptron algorithm.
* The third column contains the expected output value for the row in the training dataset.


The function takes a list of models as input; these are used to make predictions. The function
also takes a list of functions as input, one function used to make a prediction for each model.
Finally, a single row from the training dataset is included. A new row is constructed one column
at a time. Predictions are calculated using each model and the row of training data. The
expected output value from the training dataset row is then added as the last column to the
row.

In [53]:
# Make predictions with submodels and construct a new stacked row
def to_stacked_row(models, predict_list, row):
    stacked_row = list()
    for i in range(len(models)):
        prediction = predict_list[i](models[i], row)
        stacked_row.append(prediction)
    stacked_row.append(row[-1])
    return stacked_row

On some predictive modeling problems, it is possible to get an even larger boost by training
the aggregated model on both the training row and the predictions made by submodels. This
improvement gives the aggregator model the context of all the data in the training row to help
determine how and when to best combine the predictions of the submodels.

In [54]:
# Make predictions with sub-models and construct a new stacked row
def to_stacked_row(models, predict_list, row):
    stacked_row = list()
    for i in range(len(models)):
        prediction = predict_list[i](models[i], row)
        stacked_row.append(prediction)
    stacked_row.append(row[-1])
    return row[0:len(row)-1] + stacked_row

A new function name stacking() is developed. This function does 4 things:
1. It first trains a list of models (KNN and Perceptron).
2. It then uses the models to make predictions and create a new stacked dataset.
3. It then trains an aggregator model (logistic regression) on the stacked dataset.
4. It then uses the submodels and aggregator model to make predictions on the test dataset.

In [57]:
# Stacked Generalization Algorithm
def stacking(train, test):
    model_list = [knn_model, perceptron_model]
    predict_list = [knn_predict, perceptron_predict]
    models = list()
    for i in range(len(model_list)):
        model = model_list[i](train)
        models.append(model)
    stacked_dataset = list()
    for row in train:
        stacked_row = to_stacked_row(models, predict_list, row)
        stacked_dataset.append(stacked_row)
        stacked_model = logistic_regression_model(stacked_dataset)
    predictions = list()
    for row in test:
        stacked_row = to_stacked_row(models, predict_list, row)
        stacked_dataset.append(stacked_row)
        prediction = logistic_regression_predict(stacked_model, stacked_row)
        prediction = round(prediction)
        predictions.append(prediction)
    return predictions


In [60]:

# Test stacking on the sonar dataset
seed(2)
# load and prepare data
filepath = '/Users/maheshwars/Desktop/venv/data/sonar'
filename = filepath +'/sonar_data.csv'
dataset = load_csv(filename)
print('Loaded data file {0} with {1} rows and {2} columns'.format(filename, len(dataset),
                                                                  len(dataset[0])))

# convert string attributes to integers
for i in range(len(dataset[0])-1):
    str_column_to_float(dataset, i)
    
# convert class column to integers
str_column_to_int(dataset, len(dataset[0])-1)

n_folds = 3
scores = evaluate_algorithm(dataset, stacking, n_folds)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

Loaded data file /Users/maheshwars/Desktop/venv/data/sonar/sonar_data.csv with 208 rows and 61 columns
Scores: [78.26086956521739, 69.56521739130434, 42.028985507246375]
Mean Accuracy: 63.285%
